In [1]:
import scipy.stats as stat
import matplotlib.pyplot as plt
import numpy as np
import math
import itertools

In [16]:
alpha = 0.05
k = 8
u_half_alpha = -stat.norm.ppf(alpha / 2)

In [3]:
def in_range(x, interval):
    a, b = interval
    if a <= x <= b:
        return True
    return False


def pairwise(iterable):
    "s -> (s0, s1), (s1, s2), (s2, s3), ..."
    a, b = itertools.tee(iterable)
    next(b, None)
    return zip(a, b)


def group_data(data, k):
    x0 = math.floor(min(data))
    xn = math.ceil(max(data))
    bounds = [x0 + i * (xn - x0) / k for i in range(k + 1)]
    intervals = list(pairwise(bounds))
    invervals_mids = [(a + b) / 2 for (a, b) in intervals]
    grouped = np.zeros(np.size(data))
    for i in range(np.size(data)):
        entry_group = [j for j in range(k) if in_range(data[i], intervals[j])][0]
        grouped[i] = invervals_mids[entry_group]
    return grouped

def group_data_bins(data, k):
    x0 = math.floor(min(data))
    xn = math.ceil(max(data))
    bounds = [x0 + i * (xn - x0) / k for i in range(k + 1)]
    intervals = list(pairwise(bounds))
    invervals_mids = [(a + b) / 2 for (a, b) in intervals]
    grouped = np.zeros(np.size(invervals_mids))
    for i in range(np.size(data)):
        entry_group = [j for j in range(k) if in_range(data[i], intervals[j])][0]
        grouped[entry_group] += 1
    return intervals, grouped


In [17]:
def empirical_cdf(t,data):
    return sum([1 for x in data if x <= t]) / np.size(data)


In [53]:
def kolmogorov_normality_test(data):
    mean = np.mean(data)
    std = np.std(data)
    dif = [np.abs(empirical_cdf(t,data) - stat.norm.cdf(t,loc=mean,scale=std)) for t in data]
    k_statistic = np.size(data)**0.5 * np.max(dif)
    kolmogorov_quant = stat.kstwobign.ppf(1-alpha)
    p_value = 1 - stat.kstwobign.cdf(k_statistic)
    print(f'k statistic value: {k_statistic}')
    print(f'critical value: {kolmogorov_quant}')
    print(f'p-value: {p_value}')

def chi2_normality_test(data):
    mean = np.mean(data)
    std = np.std(data)
    intervals, freq = group_data_bins(data, k=10)

    exp = [np.size(data)*(stat.norm.cdf(b, loc=mean, scale=std)-stat.norm.cdf(a, loc=mean, scale=std)) for (a,b) in intervals]
    chi2 = np.sum((freq - exp)**2 / exp)

    chi2_quant = stat.chi2.ppf(1 - alpha, df=k-2) # -2 for 2 estimated parameters (location and scale)
    p_value = 1 - stat.chi2.cdf(chi2, df=k-2)
    print(f'chi2 statistic value: {chi2}')
    print(f'critical value: {chi2_quant}')
    print(f'p-value: {p_value}')

def KM_test(x, y):
    cumulated = np.append(x,y)
    dif = [np.abs(empirical_cdf(t,x) - empirical_cdf(t,y)) for t in cumulated]
    k_statistic = np.size(cumulated)**0.5 * np.max(dif)
    # TODO


    

# Simple hypothesis

## Homogeneity test for normal's expectation

$t = \frac{\overline{x_{n1}} - \overline{y_{n2}}}{s \sqrt{1/n1 + 1/n2}} \in t_{n1+n2-2}$

$p-value = P(|T| > |t|) \approx 2 F_{N(0,1)}(-|t|)$

In [4]:
n_x_gauss = 200
n_y_gauss = 300

x_gauss = stat.norm.rvs(loc=5, scale=7**0.5,size=n_x_gauss)
y_gauss = stat.norm.rvs(loc=5, scale=9**0.5,size=n_y_gauss)

s2 = ((n_x_gauss - 1) * np.var(x_gauss) + (n_y_gauss-2) * np.var(y_gauss)) / (n_x_gauss + n_y_gauss - 2)
t = (np.mean(x_gauss) - np.mean(y_gauss)) / (s2**0.5 * (1/n_x_gauss + 1/n_y_gauss)**0.5)
t_half_alpha = stat.t.ppf(1-alpha/2, df=n_x_gauss+n_y_gauss-2)
p_value = 2 * stat.norm.cdf(-np.abs(t))

print(f't statistic value: {t}')
print(f'critical values: [ -{t_half_alpha},{t_half_alpha}]')
print(f'p-value: {p_value}')

t statistic value: -0.40042628922677165
critical values: [ -1.9647389829672648,1.9647389829672648]
p-value: 0.6888425643698215


In [5]:
# # TODO: grouped
# x_grouped = group_data(x,k)
# x_uniq = np.unique(x_grouped)
# x_freq = np.array([np.count_nonzero(x_grouped == val) for val in x_uniq])
# print(*x_freq)

$x \in N(3,12)$

$y = 5x + U(-6,6)$

In [6]:
n_x_gauss = 200

x_gauss = stat.norm.rvs(loc=3, scale=12**0.5,size=n_x_gauss)
y = 5 * x_gauss + 10 * np.random.default_rng().uniform(low= -6, high=6, size=n_x_gauss)
z = y - x_gauss

t = np.mean(z) / (np.std(z) / n_x_gauss**0.5)
t_half_alpha = stat.t.ppf(1-alpha/2, df=n_x_gauss)
p_value = 2 * stat.norm.cdf(-np.abs(t))
print(f't statistic value: {t}')
print(f'critical values: [ -{t_half_alpha},{t_half_alpha}]')
print(f'p-value: {p_value}')

t statistic value: 3.822206869314888
critical values: [ -1.9718962236316089,1.9718962236316089]
p-value: 0.0001322626583037706


## poission 

In [7]:
lambda_x = 9
n_x_poisson = 200

x_poisson = stat.poisson.rvs(lambda_x, size=n_x_poisson)

x1_poisson = x_poisson[:n_x_poisson//2]
x2_poisson = x_poisson[n_x_poisson//2:]

t = (np.mean(x1_poisson) - np.mean(x2_poisson)) / (np.mean(x1_poisson) + np.mean(x2_poisson))**0.5

t_left_quant = stat.norm.ppf(alpha/2)
t_right_quant = stat.norm.ppf(1 - alpha/2)
p_value = 2 * stat.norm.cdf(-np.abs(t))

print(f't statistic value: {t}')
print(f'critical values: [ {t_left_quant},{t_right_quant}]')
print(f'p-value: {p_value}')

t statistic value: -0.03290703463055851
critical values: [ -1.9599639845400545,1.959963984540054]
p-value: 0.9737487230114853


# Homogenety tests for variance

## independent gauss data

In [8]:
n_x_gauss = 200
n_y_gauss = 300

x_gauss = stat.norm.rvs(loc=5, scale=7**0.5,size=n_x_gauss)
y_gauss = stat.norm.rvs(loc=5, scale=9**0.5,size=n_y_gauss)

fisher_left_quan = stat.f.ppf(alpha / 2, dfn = n_x_gauss-1, dfd = n_y_gauss - 1)
fisher_right_quan = stat.f.ppf(1 - alpha / 2, dfn = n_x_gauss-1, dfd = n_y_gauss - 1)

F = np.var(x_gauss) / np.var(y_gauss)
p_value = stat.f.cdf(F, dfn = n_x_gauss-1, dfd = n_y_gauss - 1)

print(f'F statistic value: {F}')
print(f'critical values: [ {fisher_left_quan},{fisher_right_quan}]')
print(f'p-value: {p_value}')

F statistic value: 0.6522853899668963
critical values: [ 0.7729893672926245,1.285204934412516]
p-value: 0.0006132508759478475


## gauss data from one sample

In [9]:
n_y_gauss = 300
y_gauss = stat.norm.rvs(loc=5, scale=9**0.5,size=n_y_gauss)
y1_gauss = y_gauss[:n_y_gauss // 2]
y2_gauss = y_gauss[n_y_gauss // 2:]

fisher_left_quan = stat.f.ppf(alpha / 2, dfn = n_y_gauss / 2 - 1, dfd = n_y_gauss / 2 - 1)
fisher_right_quan = stat.f.ppf(1 - alpha / 2, dfn = n_y_gauss / 2 - 1, dfd = n_y_gauss / 2 - 1)

F = np.var(y1_gauss) / np.var(y2_gauss)
p_value = stat.f.cdf(F, dfn = n_y_gauss / 2 - 1, dfd = n_y_gauss / 2 - 1)

print(f'F statistic value: {F}')
print(f'critical values: [ {fisher_left_quan},{fisher_right_quan}]')
print(f'p-value: {p_value}')

F statistic value: 1.2431676940473486
critical values: [ 0.7244337450679404,1.3803884852247126]
p-value: 0.9074020859070783


# Correlation test

## correlated gauss data

In [10]:
n_x_gauss = 200

x_gauss = stat.norm.rvs(loc=3, scale=12**0.5,size=n_x_gauss)
y = 5 * x_gauss + np.random.default_rng().uniform(low= -6, high=6, size=n_x_gauss)
ro, _ = stat.pearsonr(x_gauss, y)

t = ro / (1 - ro**2)**0.5 * (n_x_gauss - 2)**0.5
t_right_quant = stat.t.ppf(1-alpha/2, df=n_x_gauss)
t_left_quant = stat.t.ppf(alpha/2, df=n_x_gauss)
p_value = 2 * stat.norm.cdf(-np.abs(t))

print(f't statistic value: {t}')
print(f'critical values: [ {t_left_quant},{t_right_quant}]')
print(f'p-value: {p_value}')

t statistic value: 68.46120360234693
critical values: [ -1.9718962236316093,1.9718962236316089]
p-value: 0.0


## uncorrelated gauss data

In [11]:
n_x_gauss = 200

x_gauss = stat.norm.rvs(loc=3, scale=12**0.5,size=n_x_gauss)
x1_gauss = x_gauss[:n_x_gauss//2]
x2_gauss = x_gauss[n_x_gauss//2:]

ro, pp = stat.pearsonr(x1_gauss, x2_gauss)

t = ro / (1 - ro**2)**0.5 * (n_x_gauss//2 - 2)**0.5
t_right_quant = stat.t.ppf(1-alpha/2, df=n_x_gauss//2)
t_left_quant = stat.t.ppf(alpha/2, df=n_x_gauss//2)
p_value = 2 * stat.norm.cdf(-np.abs(t))

print(f't statistic value: {t}')
print(f'critical values: [ {t_left_quant},{t_right_quant}]')
print(f'p-value: {p_value}')

t statistic value: 0.2734188447333317
critical values: [ -1.983971518449634,1.9839715184496334]
p-value: 0.7845312684782958


# Goodness-of-fit tests

## chi square test for gauss

In [15]:
n_x_gauss = 400
# k = 10

x_gauss = stat.norm.rvs(loc=5, scale=7**0.5,size=n_x_gauss)

chi2_normality_test(x_gauss)

chi2 statistic value: 2.7901370137474406
critical value: 15.50731305586545
p-value: 0.9468293409332526


In [13]:
x_gauss_uni_noise = x_gauss + np.random.default_rng().uniform(low= -6, high=6, size=n_x_gauss)
chi2_normality_test(x_gauss_uni_noise)

chi2 statistic value: 17.915271476601298
critical value: 15.50731305586545
p-value: 0.02187075646018999


In [51]:
# noise must be small as otherwise plug-in estimations for variance will cause zero division
x_gauss_cauchy_noise = x_gauss + 0.01 * stat.cauchy.rvs(loc=0, scale=1, size=n_x_gauss)
chi2_normality_test(x_gauss_cauchy_noise)

chi2 statistic value: 4.035240533018273
critical value: 12.591587243743977
p-value: 0.6719072515169731


## Kolmogorov test

In [55]:
kolmogorov_normality_test(x_gauss)

k statistic value: 0.4714565266049764
critical value: 1.3580986393225505
p-value: 0.9793401036861186


In [56]:
kolmogorov_normality_test(x_gauss_uni_noise)

k statistic value: 1.0427705384313324
critical value: 1.3580986393225505
p-value: 0.2269417004171933


In [57]:
kolmogorov_normality_test(x_gauss_cauchy_noise)

k statistic value: 0.4426298296878606
critical value: 1.3580986393225505
p-value: 0.9895688971771405


# kolmogorov-smirnov test 